In [11]:
import cobra
import pandas as pd

import sys
sys.path.append('/home/hvdinh16/Workspace/workpy3/common/')
from custom_functions import *

from collections import OrderedDict

In [9]:
model = cobra.io.load_json_model('./model/batch/gem.json')
biom = 'BIOMASS'

#### At old GAM

In [6]:
model = cobra.io.load_json_model('./model/batch/gem.json')
model.solver = 'cplex'

model.reactions.EX_glc__D_e.bounds = (10,10)
model.objective = dict()
model.reactions.BIOMASS.objective_coefficient = 1

fba = model.optimize()
print(fba['BIOMASS'])

0.8224177919490836


In [7]:
print('NGAM:', model.reactions.ATPM_c.lower_bound)

NGAM: 1.0124


#### Find max ATP rate after reaction stoich update

In [18]:
model = cobra.io.load_json_model('./model/batch/gem.json')
model.solver = 'cplex'

model.reactions.ATPS_m.reaction = '3.0 adp_m + 10.0 h_c + 3.0 pi_m --> 3.0 atp_m + 3.0 h2o_m + 7.0 h_m'
model.reactions.FECOOR_m.reaction = '2.0 focytC_m + 4.0 h_m + 0.5 o2_m --> 2.0 ficytC_m + h2o_m + 2.0 h_c'
model.reactions.FECRq9_m.reaction = '2.0 ficytC_m + 2.0 h_m + q9h2_m --> 2.0 focytC_m + 4.0 h_c + q9_m'

# Remove GAM from biomass equation
adp = model.metabolites.adp_c
gam0 = model.reactions.get_by_id(biom).metabolites[adp]
met_dict = OrderedDict({met.id:v for met,v in model.reactions.get_by_id(biom).metabolites.items()})
for met in ['atp_c', 'h2o_c']:
    met_dict[met] += gam0
for met in ['adp_c', 'pi_c', 'h_c']:
    met_dict[met] -= gam0
biom_eqn = build_reaction_equation_from_metabolites_dict(met_dict, arrow='-->')
model.reactions.get_by_id(biom).reaction = biom_eqn

model.reactions.EX_glc__D_e.bounds = (10,10)
model.objective = dict()
model.reactions.ATPM_c.objective_coefficient = 1

model.reactions.BIOMASS.bounds = (0.8224177919490836,0.8224177919490836)

fba = model.optimize()
print(fba['ATPM_c'])

157.74795219701375


#### Calibrate

In [15]:
(131.97729983603358 - 1.0124) / 0.8224177919490836

159.24375799999922

In [19]:
(157.74795219701375 - 1.0124) / 0.8224177919490836

190.57899005997834

#### Fix equations

In [22]:
model = cobra.io.load_json_model('./model/batch/gem.json')
model.solver = 'cplex'

model.reactions.ATPS_m.reaction = '3.0 adp_m + 10.0 h_c + 3.0 pi_m --> 3.0 atp_m + 3.0 h2o_m + 7.0 h_m'
model.reactions.FECOOR_m.reaction = '2.0 focytC_m + 4.0 h_m + 0.5 o2_m --> 2.0 ficytC_m + h2o_m + 2.0 h_c'
model.reactions.FECRq9_m.reaction = '2.0 ficytC_m + 2.0 h_m + q9h2_m --> 2.0 focytC_m + 4.0 h_c + q9_m'

# Remove GAM from biomass equation
adp = model.metabolites.adp_c
gam0 = model.reactions.get_by_id(biom).metabolites[adp]
met_dict = OrderedDict({met.id:v for met,v in model.reactions.get_by_id(biom).metabolites.items()})
for met in ['atp_c', 'h2o_c']:
    met_dict[met] += gam0
for met in ['adp_c', 'pi_c', 'h_c']:
    met_dict[met] -= gam0
biom_eqn = build_reaction_equation_from_metabolites_dict(met_dict, arrow='-->')
model.reactions.get_by_id(biom).reaction = biom_eqn

# Add new GAM value
for met in ['atp_c', 'h2o_c']:
    met_dict[met] -= 190.578990
for met in ['adp_c', 'pi_c', 'h_c']:
    met_dict[met] += 190.578990
biom_eqn = build_reaction_equation_from_metabolites_dict(met_dict, arrow='-->')

In [23]:
biom_eqn

'0.684548 13BDglucan_c + 0.180144 16BDglucan_c + 0.405893 ala__L_c + 0.160363 arg__L_c + 0.192768 asn__L_c + 0.192768 asp__L_c + 190.647337 atp_c + 0.001286 ca2_c + 0.016184 chtn_c + 0.119302 ctp_c + 0.000114 cu2_c + 0.005816 cys__L_c + 0.006009 datp_c + 0.009379 dctp_c + 0.009379 dgtp_c + 0.006009 dttp_c + 0.042867 ergst_c + 0.010634 ergstest_c + 0.000664 fe2_c + 0.001922 ffabiom_c + 0.321557 gln__L_c + 0.321557 glu__L_c + 0.369334 gly_c + 0.234811 glycogen_c + 0.104534 gtp_c + 186.43364699999998 h2o_c + 0.080182 his__L_c + 0.244699 ile__L_c + 0.005176 ipcbiom_c + 0.60365 k_c + 0.332774 leu__L_c + 0.27295 lys__L_c + 0.49015 mannan_c + 0.047361 met__L_c + 0.063607 mg2_c + 9.4e-05 mn2_c + 0.007738 pail_c + 0.028783 pc_c + 0.00772 pe_c + 0.156209 phe__L_c + 0.175319 pro__L_c + 0.006643 ps_c + 0.221434 ser__L_c + 0.031223 so4_c + 0.010494 tag_c + 0.231405 thr__L_c + 0.137707 tre_c + 0.027004 trp__L_c + 0.081428 tyr__L_c + 0.062537 utp_c + 0.304524 val__L_c + 0.001576 zn2_c --> 190.57899 a